In [70]:
import glob
from itertools import product

import pandas as pd
from sourmash import signature as sig
from tqdm import tqdm

## Info about the kmer hashes loaded

In [ ]:
ksizes = 21, 27, 33, 51
ksize = ','.join(map(str, ksizes))
num_hashes = int(1e6)
dna = True
protein = True


In [1]:
output_prefix = 's3://olgabot-maca/facs/sourmash_compute_tissue_subset/'


In [2]:
! aws s3 ls $output_prefix | wc -l

99


### Copy the files to ndnd

In [3]:
! aws s3 sync $output_prefix /home/olga/scratch/maca/facs/sourmash_compute_tissue_subset/

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A1-B002764-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A1-B002764-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A1-MAA000779-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A1-MAA000779-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A1-D042253-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A1-D042253-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A10-B002775-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A10-B002775-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A10-MAA000559-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A10-MAA000559-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A10-B000971-3_39_F-1-1.sig to ../../../scra

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A18-B002764-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A18-B002764-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A18-MAA001857-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A18-MAA001857-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A20-MAA000508-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A20-MAA000508-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A2-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A2-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A2-D041914-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A2-D041914-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A2-MAA000907-3_11_M-1-1.sig to ../../../s

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B1-MAA100138-3_56_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B1-MAA100138-3_56_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B12-MAA100041-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B12-MAA100041-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B10-MAA100039-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B10-MAA100039-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B11-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B11-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B12-MAA100039-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B12-MAA100039-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B12-MAA100042-3_9_M-1-1.s

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B8-MAA000801-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B8-MAA000801-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B22-MAA001862-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B22-MAA001862-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B8-MAA000779-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B8-MAA000779-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B4-MAA100039-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B4-MAA100039-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B5-MAA100140-3_57_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B5-MAA100140-3_57_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B9-MAA100039-3_11_M-1-1.sig t

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C2-MAA000559-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C2-MAA000559-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C16-MAA000752-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C16-MAA000752-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C20-MAA000779-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C20-MAA000779-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C20-MAA001861-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C20-MAA001861-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C19-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C19-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C4-B002775-3_39_F-1-1.sig

...

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M19-MAA001868-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/M19-MAA001868-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M18-MAA000779-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/M18-MAA000779-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M17-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/M17-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M19-D042253-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/M19-D042253-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M2-B001717-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/M2-B001717-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M2-MAA000377-3_9_M-1-1.sig to ../

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N14-B002771-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N14-B002771-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N1-B000610-3_56_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N1-B000610-3_56_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N15-D041914-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N15-D041914-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N10-MAA001857-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N10-MAA001857-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N15-MAA000776-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N15-MAA000776-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N13-MAA000884-3_10_M-1-1.sig to ../..

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N9-MAA000752-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N9-MAA000752-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O1-MAA000907-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O1-MAA000907-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O1-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O1-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N8-B001750-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N8-B001750-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N9-MAA001857-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N9-MAA001857-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O10-MAA001861-3_39_F-1-1.sig to ../../.

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O5-D041914-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O5-D041914-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O6-MAA000545-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O6-MAA000545-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O2-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O2-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O5-D042253-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O5-D042253-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O6-B001717-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O6-B001717-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O18-MAA000577-3_8_M-1-1.sig to ../../../scratch/m

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P18-MAA000752-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/P18-MAA000752-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P20-D041914-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/P20-D041914-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P19-B002775-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/P19-B002775-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P2-B000610-3_56_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/P2-B000610-3_56_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P18-MAA000776-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/P18-MAA000776-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P20-MAA000910-3_10_M-1-1.sig to ../..

## Read signatures

In [1]:

folder = '/home/olga/scratch/maca/facs/sourmash_compute_tissue_subset/'

filenames = glob.iglob(f'{folder}/*.sig')


siglist = []


for filename in tqdm(filenames):
    loaded = sig.load_signatures(filename)
    siglist.extend(loaded)

1000it [1:52:22, 10.06s/it]


In [3]:
len(siglist)

6000

In [17]:
s = siglist[0]
s.name


<bound method SourmashSignature.name of SourmashSignature('A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue:NA|cell_ontology_class:bladder_urothelial_cell|free_annotation:Luminal_bladder_epithelial_cell', 0edeb71a)>

In [6]:
dir(s)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_display_name',
 '_save',
 'contained_by',
 'd',
 'jaccard',
 'md5sum',
 'minhash',
 'name',
 'similarity']

In [ ]:
s.m

In [11]:
dir(s.minhash)

['__class__',
 '__copy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'add',
 'add_hash',
 'add_many',
 'add_protein',
 'add_sequence',
 'compare',
 'contained_by',
 'copy_and_clear',
 'count_common',
 'downsample_max_hash',
 'downsample_n',
 'downsample_scaled',
 'get_hashes',
 'get_mins',
 'intersection',
 'is_molecule_type',
 'is_protein',
 'jaccard',
 'ksize',
 'max_hash',
 'merge',
 'num',
 'scaled',
 'seed',
 'set_abundances',
 'similarity',
 'similarity_ignore_maxhash',
 'subtract_mins',
 'track_abundance',
 'update']

21

In [13]:
s.minhash.is_protein

True

In [16]:
for s in siglist[:20]:
    print('---')
    print(f's.minhash.ksize: {s.minhash.ksize}')
    print(f's.minhash.is_protein: {s.minhash.is_protein}')

---
s.minhash.ksize: 21
s.minhash.is_protein: True
---
s.minhash.ksize: 21
s.minhash.is_protein: False
---
s.minhash.ksize: 33
s.minhash.is_protein: True
---
s.minhash.ksize: 33
s.minhash.is_protein: False
---
s.minhash.ksize: 51
s.minhash.is_protein: True
---
s.minhash.ksize: 51
s.minhash.is_protein: False
---
s.minhash.ksize: 21
s.minhash.is_protein: True
---
s.minhash.ksize: 21
s.minhash.is_protein: False
---
s.minhash.ksize: 33
s.minhash.is_protein: True
---
s.minhash.ksize: 33
s.minhash.is_protein: False
---
s.minhash.ksize: 51
s.minhash.is_protein: True
---
s.minhash.ksize: 51
s.minhash.is_protein: False
---
s.minhash.ksize: 21
s.minhash.is_protein: True
---
s.minhash.ksize: 21
s.minhash.is_protein: False
---
s.minhash.ksize: 33
s.minhash.is_protein: True
---
s.minhash.ksize: 33
s.minhash.is_protein: False
---
s.minhash.ksize: 51
s.minhash.is_protein: True
---
s.minhash.ksize: 51
s.minhash.is_protein: False
---
s.minhash.ksize: 21
s.minhash.is_protein: True
---
s.minhash.ksize: 2

## Read annotations

In [4]:
annotations = pd.read_csv('/home/ubuntu/tabula-muris/00_data_ingest/18_global_annotation_csv/annotations_facs.csv', 
                          index_col='cell')
annotations.index = annotations.index.str.replace('.', '-')
annotations['sample_id'] = annotations.index
annotations = annotations.fillna("NA")
print(annotations.shape)
annotations.head()

NameError: name 'pd' is not defined

In [72]:
tissues = 'Bladder', 'Kidney', 'Spleen', 'Liver', 'Pancreas'

annotations_subset = annotations.query('tissue in @tissues')
print(annotations_subset.shape)
annotations_subset.head()

(5872, 24)


,Neurog3>0_raw,Neurog3>0_scaled,cell_ontology_class,cell_ontology_id,cluster.ids,free_annotation,mouse.id,mouse.sex,plate.barcode,subsetA,...,subsetC_cluster.ids,subsetD,subsetD_cluster.ids,subsetE,subsetE_cluster.ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2,sample_id
cell,,,,,,,,,,,,,,,,,,,,,
A1-B000610-3_56_F-1-1,NA,NA,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_56_F,F,B000610,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-7.182841,-10.895094,A1-B000610-3_56_F-1-1
A1-B002764-3_38_F-1-1,NA,NA,bladder urothelial cell,CL:1001428,3,Luminal bladder epithelial cell,3_38_F,F,B002764,NA,...,NA,NA,NA,NA,NA,NA,Bladder,4.199059,-11.050240,A1-B002764-3_38_F-1-1
A1-B002771-3_39_F-1-1,NA,NA,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_39_F,F,B002771,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-11.995435,-7.325534,A1-B002771-3_39_F-1-1
A1-D041914-3_8_M-1-1,NA,NA,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_8_M,M,D041914,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-6.820022,-14.174246,A1-D041914-3_8_M-1-1
A1-D042253-3_9_M-1-1,NA,NA,bladder cell,CL:1001319,2,Bladder mesenchymal cell,3_9_M,M,D042253,NA,...,NA,NA,NA,NA,NA,NA,Bladder,-24.163538,-7.499349,A1-D042253-3_9_M-1-1


In [73]:
annotations_subsampled = annotations_subset.groupby('tissue').apply(lambda x: x.sample(200, random_state=0))
print(annotations_subsampled.shape)
annotations_subsampled.head()

(1000, 24)


Neurog3>0_raw Neurog3>0_scaled  \
tissue  cell                                                      
Bladder L4-MAA000487-3_10_M-1-1             NA               NA   
        H18-MAA000487-3_10_M-1-1            NA               NA   
        G22-D042253-3_9_M-1-1               NA               NA   
        A14-MAA000487-3_10_M-1-1            NA               NA   
        C6-D042253-3_9_M-1-1                NA               NA   

                                      cell_ontology_class cell_ontology_id  \
tissue  cell                                                                 
Bladder L4-MAA000487-3_10_M-1-1              bladder cell       CL:1001319   
        H18-MAA000487-3_10_M-1-1  bladder urothelial cell       CL:1001428   
        G22-D042253-3_9_M-1-1     bladder urothelial cell       CL:1001428   
        A14-MAA000487-3_10_M-1-1  bladder urothelial cell       CL:1001428   
        C6-D042253-3_9_M-1-1                 bladder cell       CL:1001319   

                                  cluster.ids  \
tissue  cell                                    
Bladder L4-MAA000487-3_10_M-1-1             0   
        H18-MAA000487-3_10_M-1-1            6   
        G22-D042253-3_9_M-1-1               1   
        A14-MAA000487-3_10_M-1-1            1   
        C6-D042253-3_9_M-1-1                2   

                                                  free_annotation mouse.id  \
tissue  cell                                                                 
Bladder L4-MAA000487-3_10_M-1-1          Bladder mesenchymal cell   3_10_M   
        H18-MAA000487-3_10_M-1-1    Basal bladder epithelial cell   3_10_M   
        G22-D042253-3_9_M-1-1     Luminal bladder epithelial cell    3_9_M   
        A14-MAA000487-3_10_M-1-1  Luminal bladder epithelial cell   3_10_M   
        C6-D042253-3_9_M-1-1             Bladder mesenchymal cell    3_9_M   

                                 mouse.sex plate.barcode subsetA  \
tissue  cell                                                       
Bladder L4-MAA000487-3_10_M-1-1          M     MAA000487      NA   
        H18-MAA000487-3_10_M-1-1         M     MAA000487      NA   
        G22-D042253-3_9_M-1-1            M       D042253      NA   
        A14-MAA000487-3_10_M-1-1         M     MAA000487      NA   
        C6-D042253-3_9_M-1-1             M       D042253      NA   

                                            ...             \
tissue  cell                                ...              
Bladder L4-MAA000487-3_10_M-1-1             ...              
        H18-MAA000487-3_10_M-1-1            ...              
        G22-D042253-3_9_M-1-1               ...              
        A14-MAA000487-3_10_M-1-1            ...              
        C6-D042253-3_9_M-1-1                ...              

                                 subsetC_cluster.ids subsetD  \
tissue  cell                                                   
Bladder L4-MAA000487-3_10_M-1-1                   NA      NA   
        H18-MAA000487-3_10_M-1-1                  NA      NA   
        G22-D042253-3_9_M-1-1                     NA      NA   
        A14-MAA000487-3_10_M-1-1                  NA      NA   
        C6-D042253-3_9_M-1-1                      NA      NA   

                                 subsetD_cluster.ids subsetE  \
tissue  cell                                                   
Bladder L4-MAA000487-3_10_M-1-1                   NA      NA   
        H18-MAA000487-3_10_M-1-1                  NA      NA   
        G22-D042253-3_9_M-1-1                     NA      NA   
        A14-MAA000487-3_10_M-1-1                  NA      NA   
        C6-D042253-3_9_M-1-1                      NA      NA   

                                 subsetE_cluster.ids subtissue   tissue  \
tissue  cell                                                              
Bladder L4-MAA000487-3_10_M-1-1                   NA        NA  Bladder   
        H18-MAA000487-3_10_M-1-1                  NA        NA  Bladder   
        G22-D04

In [74]:
annotations_subsampled.groupby(['cell_ontology_class']).size()

cell_ontology_class
B cell                                    163
Kupffer cell                               14
T cell                                     44
bladder cell                              105
bladder urothelial cell                    95
endothelial cell                           64
endothelial cell of hepatic sinusoid       49
epithelial cell of proximal tubule         87
hepatocyte                                117
kidney collecting duct epithelial cell     34
leukocyte                                  12
macrophage                                 23
natural killer cell                         8
pancreatic A cell                          49
pancreatic D cell                          17
pancreatic PP cell                          6
pancreatic acinar cell                     25
pancreatic ductal cell                     15
pancreatic stellate cell                    5
type B pancreatic cell                     68
dtype: int64

In [75]:
sample_id_with_annotation = \
    annotations['sample_id'] + '|tissue:' + annotations['tissue'] \
    + '|subtissue:' + annotations['subtissue'] \
    + '|cell_ontology_class:' + annotations['cell_ontology_class'] \
    + '|free_annotation:' + annotations['free_annotation']
sample_id_with_annotation = sample_id_with_annotation.str.replace(' ', '_')
sample_id_with_annotation.name = 'sample_id_with_annotation'
sample_id_with_annotation.head()

cell
A1-B000610-3_56_F-1-1    A1-B000610-3_56_F-1-1|tissue:Bladder|subtissue...
A1-B002764-3_38_F-1-1    A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue...
A1-B002771-3_39_F-1-1    A1-B002771-3_39_F-1-1|tissue:Bladder|subtissue...
A1-D041914-3_8_M-1-1     A1-D041914-3_8_M-1-1|tissue:Bladder|subtissue:...
A1-D042253-3_9_M-1-1     A1-D042253-3_9_M-1-1|tissue:Bladder|subtissue:...
Name: sample_id_with_annotation, dtype: object

In [63]:
# ! aws s3 ls s3://olgabot-maca/facs/

In [64]:
# ! aws s3 ls s3://olgabot-maca/facs/

In [65]:
s3_prefix = 's3://czbiohub-maca/'

maca_aws = pd.read_table('s3://olgabot-maca/facs/aws_maca_remux_bytes_fastq.txt', sep='\s+', header=None, 
                         names=['date', 'time', 'filesize', 'suffix'])
maca_aws['megabytes'] = maca_aws['filesize']/1e6
maca_aws['novaseq_run'] = maca_aws['suffix'].str.extract('([AB]\d{5})')
maca_aws['novaseq_run_number'] = maca_aws['novaseq_run'].str.lstrip("AB")
maca_aws['sample_id'] = maca_aws['suffix'].str.split('/').str[-2]
maca_aws['sample_id_no_sample_number'] = maca_aws.sample_id.map(lambda x: '_'.join(x.split('_')[:3]))
maca_aws = maca_aws.join(sample_id_with_annotation, on='sample_id_no_sample_number')
maca_aws['full_s3_path'] = s3_prefix + maca_aws['suffix']
maca_aws['read'] = maca_aws['suffix'].str.extract('_(R[12])_')
print(maca_aws.shape)

maca_aws.head()

(263160, 12)


,date,time,filesize,suffix,megabytes,novaseq_run,novaseq_run_number,sample_id,sample_id_no_sample_number,sample_id_with_annotation,full_s3_path,read
0,2017-10-31,19:17:18,52570660,remux_data/170907_A00111_0051_BH2HWLDMXX/rawda...,52.570660,A00111,00111,A1-D041911-3_8_M-1-1_S128,A1-D041911-3_8_M-1-1,A1-D041911-3_8_M-1-1|tissue:Tongue|subtissue:N...,s3://czbiohub-maca/remux_data/170907_A00111_00...,R1
1,2017-10-31,19:17:18,55481347,remux_data/170907_A00111_0051_BH2HWLDMXX/rawda...,55.481347,A00111,00111,A1-D041911-3_8_M-1-1_S128,A1-D041911-3_8_M-1-1,A1-D041911-3_8_M-1-1|tissue:Tongue|subtissue:N...,s3://czbiohub-maca/remux_data/170907_A00111_00...,R2
2,2017-10-31,19:14:53,53806,remux_data/170907_A00111_0051_BH2HWLDMXX/rawda...,0.053806,A00111,00111,A1-D041912-3_8_M-1-1_S1,A1-D041912-3_8_M-1-1,A1-D041912-3_8_M-1-1|tissue:Marrow|subtissue:G...,s3://czbiohub-maca/remux_data/170907_A00111_00...,R1
3,2017-10-31,19:14:53,54967,remux_data/170907_A00111_0051_BH2HWLDMXX/rawda...,0.054967,A00111,00111,A1-D041912-3_8_M-1-1_S1,A1-D041912-3_8_M-1-1,A1-D041912-3_8_M-1-1|tissue:Marrow|subtissue:G...,s3://czbiohub-maca/remux_data/170907_A00111_00...,R2
4,2017-10-31,19:14:53,36204,remux_data/170907_A00111_0051_BH2HWLDMXX/rawda...,0.036204,A00111,00111,A1-D041914-3_8_M-1-1_S269,A1-D041914-3_8_M-1-1,A1-D041914-3_8_M-1-1|tissue:Bladder|subtissue:...,s3://czbiohub-maca/remux_data/170907_A00111_00...,R1


In [66]:
maca_aws_subsampled = maca_aws.query('sample_id_no_sample_number in @annotations_subsampled.sample_id')
print(maca_aws_subsampled.shape)
maca_aws_subsampled.head()

(4250, 12)


,date,time,filesize,suffix,megabytes,novaseq_run,novaseq_run_number,sample_id,sample_id_no_sample_number,sample_id_with_annotation,full_s3_path,read
10,2017-10-31,19:32:42,25187121,remux_data/170907_A00111_0051_BH2HWLDMXX/rawda...,25.187121,A00111,00111,A1-D042253-3_9_M-1-1_S43,A1-D042253-3_9_M-1-1,A1-D042253-3_9_M-1-1|tissue:Bladder|subtissue:...,s3://czbiohub-maca/remux_data/170907_A00111_00...,R1
11,2017-10-31,19:32:42,26936765,remux_data/170907_A00111_0051_BH2HWLDMXX/rawda...,26.936765,A00111,00111,A1-D042253-3_9_M-1-1_S43,A1-D042253-3_9_M-1-1,A1-D042253-3_9_M-1-1|tissue:Bladder|subtissue:...,s3://czbiohub-maca/remux_data/170907_A00111_00...,R2
56,2017-10-31,19:14:53,45884,remux_data/170907_A00111_0051_BH2HWLDMXX/rawda...,0.045884,A00111,00111,A10-D041914-3_8_M-1-1_S279,A10-D041914-3_8_M-1-1,A10-D041914-3_8_M-1-1|tissue:Bladder|subtissue...,s3://czbiohub-maca/remux_data/170907_A00111_00...,R1
57,2017-10-31,19:14:53,46591,remux_data/170907_A00111_0051_BH2HWLDMXX/rawda...,0.046591,A00111,00111,A10-D041914-3_8_M-1-1_S279,A10-D041914-3_8_M-1-1,A10-D041914-3_8_M-1-1|tissue:Bladder|subtissue...,s3://czbiohub-maca/remux_data/170907_A00111_00...,R2
72,2017-10-31,19:15:21,53760,remux_data/170907_A00111_0051_BH2HWLDMXX/rawda...,0.053760,A00111,00111,A10-MAA000559-3_8_M-1-1_S17,A10-MAA000559-3_8_M-1-1,A10-MAA000559-3_8_M-1-1|tissue:Spleen|subtissu...,s3://czbiohub-maca/remux_data/170907_A00111_00...,R1


In [100]:
samples = maca_aws_subsampled.groupby(['sample_id_no_sample_number', 'read']).apply(lambda x: '|'.join(x['full_s3_path'])).unstack()
samples = samples.join(sample_id_with_annotation)
samples = samples.rename(columns={'sample_id_with_annotation': 'name', "R1": "read1", "R2": "read2"})
print(samples.shape)
samples.head()

(1000, 3)


,read1,read2,name
sample_id_no_sample_number,,,
A1-B002764-3_38_F-1-1,s3://czbiohub-maca/remux_data/170914_A00111_00...,s3://czbiohub-maca/remux_data/170914_A00111_00...,A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue...
A1-D042253-3_9_M-1-1,s3://czbiohub-maca/remux_data/170907_A00111_00...,s3://czbiohub-maca/remux_data/170907_A00111_00...,A1-D042253-3_9_M-1-1|tissue:Bladder|subtissue:...
A1-MAA000487-3_10_M-1-1,s3://czbiohub-maca/remux_data/170910_A00111_00...,s3://czbiohub-maca/remux_data/170910_A00111_00...,A1-MAA000487-3_10_M-1-1|tissue:Bladder|subtiss...
A1-MAA000779-3_11_M-1-1,s3://czbiohub-maca/remux_data/170928_A00111_00...,s3://czbiohub-maca/remux_data/170928_A00111_00...,A1-MAA000779-3_11_M-1-1|tissue:Spleen|subtissu...
A1-MAA100039-3_11_M-1-1,s3://czbiohub-maca/remux_data/170928_A00111_00...,s3://czbiohub-maca/remux_data/170928_A00111_00...,A1-MAA100039-3_11_M-1-1|tissue:Liver|subtissue...


In [101]:
! aws s3 ls s3://olgabot-maca/facs/

                           PRE merge_tabula_muris/
                           PRE sourmash/
                           PRE sourmash_compare/
                           PRE sourmash_compare_combined/
                           PRE sourmash_compare_no_track_abundance/
                           PRE sourmash_compare_no_track_abundance_combined/
                           PRE sourmash_compute_all/
                           PRE sourmash_compute_all_b_cells/
                           PRE sourmash_dna-only_trim=false_scaled=100/
                           PRE sourmash_dna-only_trim=false_scaled=1000/
                           PRE sourmash_dna-only_trim=false_scaled=1100/
                           PRE sourmash_dna-only_trim=false_scaled=1200/
                           PRE sourmash_dna-only_trim=false_scaled=1300/
                           PRE sourmash_dna-only_trim=false_scaled=1400/
                           PRE sourmash_dna-only_trim=false_scaled=1500/
                           PRE s

In [102]:
# ! aws s3 ls s3://olgabot-maca/facs/sourmash/

In [103]:

samples['output'] = output_prefix +  samples.index + '.sig'
samples.head()

,read1,read2,name,output
sample_id_no_sample_number,,,,
A1-B002764-3_38_F-1-1,s3://czbiohub-maca/remux_data/170914_A00111_00...,s3://czbiohub-maca/remux_data/170914_A00111_00...,A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue...,s3://olgabot-maca/facs/sourmash_compute_tissue...
A1-D042253-3_9_M-1-1,s3://czbiohub-maca/remux_data/170907_A00111_00...,s3://czbiohub-maca/remux_data/170907_A00111_00...,A1-D042253-3_9_M-1-1|tissue:Bladder|subtissue:...,s3://olgabot-maca/facs/sourmash_compute_tissue...
A1-MAA000487-3_10_M-1-1,s3://czbiohub-maca/remux_data/170910_A00111_00...,s3://czbiohub-maca/remux_data/170910_A00111_00...,A1-MAA000487-3_10_M-1-1|tissue:Bladder|subtiss...,s3://olgabot-maca/facs/sourmash_compute_tissue...
A1-MAA000779-3_11_M-1-1,s3://czbiohub-maca/remux_data/170928_A00111_00...,s3://czbiohub-maca/remux_data/170928_A00111_00...,A1-MAA000779-3_11_M-1-1|tissue:Spleen|subtissu...,s3://olgabot-maca/facs/sourmash_compute_tissue...
A1-MAA100039-3_11_M-1-1,s3://czbiohub-maca/remux_data/170928_A00111_00...,s3://czbiohub-maca/remux_data/170928_A00111_00...,A1-MAA100039-3_11_M-1-1|tissue:Liver|subtissue...,s3://olgabot-maca/facs/sourmash_compute_tissue...


In [106]:

samples['ksize'] = ksize
samples['num_hashes'] = num_hashes
samples['dna'] = dna
samples['protein'] = protein
samples.index.name = 'id'
samples['trim_low_abundance_kmers'] = False
samples.head()

,read1,read2,name,output,ksize,num_hashes,dna,protein,trim_low_abundance_kmers
id,,,,,,,,,
A1-B002764-3_38_F-1-1,s3://czbiohub-maca/remux_data/170914_A00111_00...,s3://czbiohub-maca/remux_data/170914_A00111_00...,A1-B002764-3_38_F-1-1|tissue:Bladder|subtissue...,s3://olgabot-maca/facs/sourmash_compute_tissue...,"21,27,33,51",1000000,True,True,False
A1-D042253-3_9_M-1-1,s3://czbiohub-maca/remux_data/170907_A00111_00...,s3://czbiohub-maca/remux_data/170907_A00111_00...,A1-D042253-3_9_M-1-1|tissue:Bladder|subtissue:...,s3://olgabot-maca/facs/sourmash_compute_tissue...,"21,27,33,51",1000000,True,True,False
A1-MAA000487-3_10_M-1-1,s3://czbiohub-maca/remux_data/170910_A00111_00...,s3://czbiohub-maca/remux_data/170910_A00111_00...,A1-MAA000487-3_10_M-1-1|tissue:Bladder|subtiss...,s3://olgabot-maca/facs/sourmash_compute_tissue...,"21,27,33,51",1000000,True,True,False
A1-MAA000779-3_11_M-1-1,s3://czbiohub-maca/remux_data/170928_A00111_00...,s3://czbiohub-maca/remux_data/170928_A00111_00...,A1-MAA000779-3_11_M-1-1|tissue:Spleen|subtissu...,s3://olgabot-maca/facs/sourmash_compute_tissue...,"21,27,33,51",1000000,True,True,False
A1-MAA100039-3_11_M-1-1,s3://czbiohub-maca/remux_data/170928_A00111_00...,s3://czbiohub-maca/remux_data/170928_A00111_00...,A1-MAA100039-3_11_M-1-1|tissue:Liver|subtissue...,s3://olgabot-maca/facs/sourmash_compute_tissue...,"21,27,33,51",1000000,True,True,False
